In [24]:
import random
import datetime
from subprocess import check_output, PIPE, Popen
import shlex

class Gateway():
    address = ""
    latency = 0.0
    ts = None
    status = True
    def __init__(self, address, latency, ts, status = True):
        self.address = address
        self.latency = latency
        self.ts = ts
        self.status = status
        
    def setStatus(self, status):
        self.status=status
    
    def getStatus(self):
        return self.status
    
    def changeInformation(self, latency, ts):
        self.latency = latency
        self.ts = ts
    
    def getLatency(self):
        return self.latency
    
    def getTimestamp(self):
        return self.ts

class Node():
    user = 'david.pinilla'
    password = \
    address = ""
    neighbors = []
    def __init__(self, address, neighbors = None):
        self.address = address
        self.neighbors = neighbors
        
    def ping(self):
        self.ts = datetime.datetime.now()
        if len(self.neighbors) > 2:
            rand_choice = random.sample(set(self.neighbors),2)
            for rand in rand_choice:                
                cmd='''curl -x '''+rand.address+''':3128 -U david.pinilla:"|Jn 5DJ\\7inbNniK|m@^ja&>C" -m 180 -w %{time_total} http://ovh.net/files/1Mb.dat -o /dev/null -s'''
                command = Popen(shlex.split(cmd),stdout=PIPE, stderr=PIPE)
                stdout, stderr = command.communicate()
                print(float(stdout))
                


# Client node information    
class Client():
    address = ""
    neighbours = []
    gateways = []
    
    def __init__(self, address, neighbours = [], gateways = []):
        self.address = address
        self.neighbours = neighbours
        self.gateways = gateways
    
    def addNeighbour(self, neighbour):
        self.neighbours.append(neighbour)
    
    def removeNeighbour(self, neighbour):
        self.neighbours.remove(neighbour)
    
    def addGateway(self, gateway):
        self.gateways.append(gateway)
    
    def removeGateway(self, gateway):
        self.gateways.remove(gateway)
    
    def pingGateway(self, gateway):        
        cmd='''curl -x '''+gateway.address+''':3128 -U david.pinilla:"|Jn 5DJ\\7inbNniK|m@^ja&>C" -m 180 -w %{time_total} http://ovh.net/files/1Mb.dat -o /dev/null -s'''
        command = Popen(shlex.split(cmd),stdout=PIPE, stderr=PIPE)
        stdout, stderr = command.communicate()
        gateway.ts = datetime.datetime.now()
        gateway.latency = float(stdout)        
        return gateway
    
    def select2Random(self):
        return random.sample(set([gw for gw in self.gateways if gw.status is True]), 2)
    
    def selectRandomize(self):
        selected = None
        if len(self.gateways) > 2:
            rand_choice = self.select2Random()
            if rand_choice[0].latency > rand_choice[1].latency:
                selected = rand_choice[1]
            else:
                selected = rand_choice[0]
        else:
            selected = self.gateways[0]
        return selected
    
    def selectBest(self):
        selected = None
        for gw in self.gateways:
            if selected is None:
                selected = gw
                continue
            if gw.latency < selected.latency:
                selected = gw
                
        return selected
                
node1 = Gateway('10.139.40.85', 0.15, '2018 Feb 15 18:59:15')
node2 = Gateway('10.139.40.122', 0.20, '2018 Feb 15 18:20:15',False)
node3 = Gateway('10.138.57.2', 0.02, '2018 Feb 15 18:45:07')
node4 = Gateway('10.228.0.83', 0.5, '2018 Feb 15 18:47:21', False)
node5 = Gateway('10.138.77.2', 0.78, '2018 Feb 15 18:26:19')
node6 = Gateway('10.138.85.130', 0.35, '2018 Feb 15 19:00:01')
node7 = Gateway('10.139.17.4', 2.1, '2018 Feb 15 18:26:04', False)
node8 = Gateway('10.139.37.194', 1.5, '2018 Feb 15 18:30:14')
node9 = Gateway('10.138.62.2', 0.04, '2018 Feb 15 19:47:59')
node10 = Gateway('10.138.25.67', 0.09, '2018 Feb 15 17:59:07')


#client1= Node('10.1.9.75')
#client2= Node('10.139.40.115',[node3,node2,node6, node5])
#client3= Node('10.139.40.116')
client4= Client('10.139.40.77', [node1,node2,node4, node8, node9, node10, node6,node7, node5, node3])

print(client4.pingGateway(node9).latency)
#client4.ping()
        

129.33553
